In [1]:
import torch
import numpy as np

In [2]:
input_images = torch.tensor(
      [[[[0,  1,  2],
         [3,  4,  5],
         [6,  7,  8]],

        [[9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]],


       [[[27, 28, 29],
         [30, 31, 32],
         [33, 34, 35]],

        [[36, 37, 38],
         [39, 40, 41],
         [42, 43, 44]],

        [[45, 46, 47],
         [48, 49, 50],
         [51, 52, 53]]]])


def get_padding2d(input_images):
    padded_images = torch.nn.functional.pad(input_images, [1,1,1,1]).float()
    return padded_images


correct_padded_images = torch.tensor(
       [[[[0.,  0.,  0.,  0.,  0.],
          [0.,  0.,  1.,  2.,  0.],
          [0.,  3.,  4.,  5.,  0.],
          [0.,  6.,  7.,  8.,  0.],
          [0.,  0.,  0.,  0.,  0.]],

         [[0.,  0.,  0.,  0.,  0.],
          [0.,  9., 10., 11.,  0.],
          [0., 12., 13., 14.,  0.],
          [0., 15., 16., 17.,  0.],
          [0.,  0.,  0.,  0.,  0.]],

         [[0.,  0.,  0.,  0.,  0.],
          [0., 18., 19., 20.,  0.],
          [0., 21., 22., 23.,  0.],
          [0., 24., 25., 26.,  0.],
          [0.,  0.,  0.,  0.,  0.]]],


        [[[0.,  0.,  0.,  0.,  0.],
          [0., 27., 28., 29.,  0.],
          [0., 30., 31., 32.,  0.],
          [0., 33., 34., 35.,  0.],
          [0.,  0.,  0.,  0.,  0.]],

         [[0.,  0.,  0.,  0.,  0.],
          [0., 36., 37., 38.,  0.],
          [0., 39., 40., 41.,  0.],
          [0., 42., 43., 44.,  0.],
          [0.,  0.,  0.,  0.,  0.]],

         [[0.,  0.,  0.,  0.,  0.],
          [0., 45., 46., 47.,  0.],
          [0., 48., 49., 50.,  0.],
          [0., 51., 52., 53.,  0.],
          [0.,  0.,  0.,  0.,  0.]]]])

In [3]:
print(torch.allclose(get_padding2d(input_images), correct_padded_images))

True


# Формула размерности выхода свёрточного слоя

In [4]:
def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width

In [5]:
print(np.array_equal(
    calc_out_shape(input_matrix_shape=[2, 3, 10, 10],
                   out_channels=10,
                   kernel_size=3,
                   stride=1,
                   padding=0),
    [2, 10, 8, 8]))

True


# Код для проверки

In [6]:
import torch
from abc import ABC, abstractmethod

# абстрактный класс для свёрточного слоя
class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass

In [7]:
# класс-обёртка над torch.nn.Conv2d для унификации интерфейса
class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels,
                                      out_channels,
                                      kernel_size,
                                      stride,
                                      padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)

In [8]:
# функция, создающая объект класса cls и возвращающая свёртку от input_matrix
def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)

In [9]:
# Функция, тестирующая класс conv2d_cls.
# Возвращает True, если свёртка совпадает со свёрткой с помощью torch.nn.Conv2d.
def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
        [[[[0., 1, 0],
           [1,  2, 1],
           [0,  1, 0]],

          [[1, 2, 1],
           [0, 3, 3],
           [0, 1, 10]],

          [[10, 11, 12],
           [13, 14, 15],
           [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor())\
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    return torch.allclose(custom_conv2d_out, conv2d_out)\
           and (custom_conv2d_out.shape == conv2d_out.shape)

    print(test_conv2d_layer(Conv2d))

# Свёрточный слой

Рассмотрим свёртку пакета из
– однослойного изображения 3х3
– с ядром из одного фильтра 2х2,
– stride = 1, то есть,
на выходе должна получиться одна матрица 2х2.

Строго записанная размерность выхода равна (1 – изображений в пакете, 1 – количество фильтров в ядре, 2 – высота матрицы выхода, 2 – ширина матрицы выхода).

Пусть W – веса ядра, X – вход, Y – выход.

На каждой итерации цикла фильтр умножается попиксельно на часть изображения, а потом 4 получившиеся числа складываются – получается один пиксель выхода.

Требуемое количество итераций для данного случая – 4, так как может быть 2 положения ядра и 2 по вертикали, общее число итераций – произведение количеств положений, то есть в данном случае 2х2 = 4.

Давайте перейдём от простого случая к общему.

– Если бы изображение было многослойным, например трёхслойное – RGB, значит, фильтры в ядре тоже должны быть трёхслойные. Каждый слой фильтра попиксельно умножается на соответствующий слой исходного изображения. То есть в данном случае после умножения получилось бы 4х3 = 12 произведений, результаты которых складываются, и получается значение выходного пикселя.
– Если бы фильтров в ядре было больше одного, то добавился бы внешний цикл по фильтрам, внутри которого мы считаем свёртку для каждого фильтра.
– Если бы во входном пакете было более 1 изображения, то добавился бы ещё один внешний цикл по изображениям в пакете.

Во всех шагах этого урока мы считаем bias в сверточных слоях нулевым.

In [ ]:
# свёрточный слой через циклы:
class Conv2dLoop(ABCConv2d):
    def __call__(self, input_tensor):
        batch_size, out_channels, output_height, output_width = \
            calc_out_shape(
                input_tensor.shape,
                self.out_channels,
                self.kernel_size,
                self.stride,
                padding=0
            )

        # создадим выходной тензор, заполненный нулями:
        output_tensor = torch.zeros(
            batch_size,
            out_channels,
            output_height,
            output_width
        )

        # цикл по входным пакетам
        for num_batch, batch in enumerate(input_tensor):

            # цикл по фильтрам (количество фильтров совпадает с количеством выходных каналов)
            for num_kernel, kernel in enumerate(self.kernel):

                # цикл по размерам выходного изображения
                for height in range(output_height):
                    for width in range(output_width):

                        # вырезаем кусочек из одного пакета (сразу по всем входным каналам)
                        row = self.stride*height
                        col = self.stride*width
                        slice = batch[:, row:row + self.kernel_size, col:col + self.kernel_size]

                        # умножаем кусочек на фильтр
                        res = float((slice * kernel).sum())

                        # заполняем ячейку в выходном тензоре
                        output_tensor[num_batch, num_kernel, height, width] = res

        return output_tensor

# Корректность реализации определится в сравнии со стандартным слоем из pytorch.
# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
# print(test_conv2d_layer(Conv2dLoop))